In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip install .
#!pip install -e .

In [ ]:
!pip install -r ./examples/pytorch/text-classification/requirements.txt

In [ ]:
# When installing in editable mode, `transformers` is not recognized as a package.
# this line must be added in order for python to be aware of transformers.
#RUN cd transformers && python3 setup.py develop
#%cd /content/transformers 
!python setup.py develop

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch
import numpy as np

n_layers, n_heads = model.config.num_hidden_layers, model.config.num_attention_heads
head_mask = (torch.randint(0,101,(n_layers, n_heads)) > 50) *1

#we need at least one alive head at layer
for i in np.where(np.all(np.isclose(head_mask, 0), axis=1))[0]:
    head_mask[i][0] = 1.0

In [ ]:
head_mask

tensor([[1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
        [0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1],
        [1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1],
        [1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0]])

In [ ]:
heads_to_prune = dict(
        (layer, torch.atleast_1d((1 - head_mask[layer].long()).nonzero().squeeze()).tolist()) for layer in range(len(head_mask))
    )
heads_to_prune

{0: [1, 4, 6, 7, 10, 11],
 1: [0, 6, 7, 8, 10, 11],
 2: [3, 4, 5, 10],
 3: [0, 1, 9, 10, 11],
 4: [0, 2, 4, 7],
 5: [0, 3, 7, 8, 10, 11],
 6: [0, 2, 3, 6, 8],
 7: [2, 4, 9, 10, 11],
 8: [3, 5, 6, 8],
 9: [3, 6, 7],
 10: [3, 4, 5, 7, 8, 9],
 11: [6, 10, 11]}

In [ ]:
model.get_memory_footprint() / 1024 / 1024

417.6494140625

In [ ]:
model.prune_heads(heads_to_prune)

In [ ]:
model.get_memory_footprint() / 1024 / 1024

374.857666015625

In [ ]:
%cd /content/

/content


In [ ]:
!python ./run_glue_prun_final.py \
  --model_name_or_path bert-base-cased \
  --task_name stsb \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 64 \
  --per_device_eval_batch_size 256 \
  --learning_rate 5e-5 \
  --num_train_epochs 3 \
  --seed 1337 \
  --run_name "test_32_off" \
  --output_dir "glue_bert_prune" \
  --try_masking \
  --masking_threshold 0.9 \
  --masking_amount 0.05

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
INFO:__main__:Training/evaluation parameters PruneTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
dont_normalize_global_importance=False,
dont_normalize_importance_by_layer=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('/content/glue_bert_prune')
model = AutoModel.from_pretrained("/content/glue_bert_prune")

Some weights of the model checkpoint at /content/glue_bert_prune were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
masked_heads = torch.tensor(np.load('/content/glue_bert_prune/head_mask.npy'))

#we need at least one alive head at layer
for i in np.where(np.all(np.isclose(masked_heads, 0), axis=1))[0]:
    masked_heads[i][0] = 1.0

In [ ]:
heads_to_prune = dict(
        (layer, torch.atleast_1d((1 - head_mask[layer].long()).nonzero().squeeze()).tolist()) for layer in range(len(head_mask))
    )
heads_to_prune

{0: [1, 4, 6, 7, 10, 11],
 1: [0, 6, 7, 8, 10, 11],
 2: [3, 4, 5, 10],
 3: [0, 1, 9, 10, 11],
 4: [0, 2, 4, 7],
 5: [0, 3, 7, 8, 10, 11],
 6: [0, 2, 3, 6, 8],
 7: [2, 4, 9, 10, 11],
 8: [3, 5, 6, 8],
 9: [3, 6, 7],
 10: [3, 4, 5, 7, 8, 9],
 11: [6, 10, 11]}

In [ ]:
model.get_memory_footprint() / 1024 / 1024

413.1787109375

In [ ]:
model.prune_heads(heads_to_prune)

In [ ]:
model.get_memory_footprint() / 1024 / 1024

370.386962890625

In [ ]:
model.save_pretrained('pruned_bert')
tokenizer.save_pretrained('pruned_bert')

('pruned_bert/tokenizer_config.json',
 'pruned_bert/special_tokens_map.json',
 'pruned_bert/vocab.txt',
 'pruned_bert/added_tokens.json',
 'pruned_bert/tokenizer.json')

In [ ]:
!python ./run_glue_prun_final.py \
  --model_name_or_path pruned_bert \
  --task_name stsb \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 64 \
  --per_device_eval_batch_size 256 \
  --learning_rate 5e-5 \
  --num_train_epochs 3 \
  --seed 1337 \
  --run_name "test_32_off" \
  --output_dir "glue_bert_prune" 

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
INFO:__main__:Training/evaluation parameters PruneTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
dont_normalize_global_importance=False,
dont_normalize_importance_by_layer=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinis